# Prepare Dataset

### GDrive setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd 'drive/My Drive/Colab Notebooks/4th year project/OULAD dataset/Deadline'

/content/drive/My Drive/Colab Notebooks/4th year project/OULAD dataset/Deadline


In [ ]:
ls

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
assessments_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/assessments.csv')
courses_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/courses.csv')
student_assessments_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/studentAssessment.csv')
student_info_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/studentInfo.csv')
student_registration_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/studentRegistration.csv')
student_vle_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/studentVle.csv')
vle_df = pd.read_csv('../../OULAD dataset/anonymisedOUDataSet/vle.csv')

sa_days_deadline_df = pd.read_csv('../../OULAD dataset/StudentAssessmentDaysDeadline.csv')

sns.set(rc={'figure.figsize':(11.7,8.27)})

### Dates for each assessment

In [ ]:
assessments_df.loc[(assessments_df['code_module']=="AAA") & (assessments_df['code_presentation']=="2013J")].date

0     19.0
1     54.0
2    117.0
3    166.0
4    215.0
5      NaN
Name: date, dtype: float64

In [ ]:
DEADLINE = 220

SCORE_DEADLINE = DEADLINE
CLICKS_DEADLINE = DEADLINE

### Join assessment data with individual student assessments

In [ ]:
def get_score_data(assessments_df, student_assessments_df, score_deadline):
  # drop columns not required to join with student_assessments_df
  assessments_df.drop(['weight'], axis = 1, inplace = True)
  assessments_df.code_module.value_counts()

  # merge each student assessments with assessment data
  combined_assessment = pd.merge(student_assessments_df,assessments_df,on='id_assessment')
  combined_assessment.drop(['is_banked','date_submitted'],axis = 1,inplace=True)

  # Get all assessments which are not the final exam and happened before a certain date
  combined_assessment = combined_assessment[(combined_assessment['assessment_type'] != 'Exam') & (combined_assessment['date'] <= score_deadline)]

  grouped_student_assessments = combined_assessment.groupby(['id_student','code_module','code_presentation']).mean()
  grouped_student_assessments.drop(['id_assessment', 'date'],axis=1,inplace = True)

  return grouped_student_assessments

In [ ]:
def merge_df(student_info_df, other_df, on, how='inner'):
  student_all_info = pd.merge(student_info_df,other_df,on=on, how=how)

  return student_all_info

In [ ]:
score_df = get_score_data(assessments_df.copy(), student_assessments_df.copy(), SCORE_DEADLINE)

score_df.head()

score
id_student code_module code_presentation           
6516       AAA         2014J              61.800000
8462       DDD         2013J              87.666667
                       2014J              86.500000
11391      AAA         2013J              82.000000
23629      BBB         2013B              82.500000

### Merge score_df and all other student_info

In [ ]:
student_all_info = merge_df(student_info_df.copy(), score_df.copy(), on='id_student')

student_all_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,score
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,82.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,66.4
2,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,76.0
3,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,54.4
4,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,68.0


### Get the number of clicks by a deadline

In [ ]:
# click data
def inner_merge(left_df,right_df,right_cols,on_cols):
    right_df = right_df[right_cols]
    left_df = left_df.merge(right_df, on = on_cols)
    return left_df.drop_duplicates()


def get_click_data(student_vle_df,vle_df,click_deadline):

  def clicks_from(clicks, deadline):
    temp = clicks[clicks['date'] <= deadline]
    temp = temp.drop('date', axis = 1)
    temp = temp.groupby(['code_module','code_presentation','id_student','activity_type']).sum()
    temp = temp.rename(columns = {'sum_click': f'sum_click{deadline} sum'})
    temp = temp.reset_index() # this fills out the missing columns for merging later 
    return temp

  clicks = inner_merge(student_vle_df,
                        vle_df,
                        ['id_site','code_module','code_presentation','activity_type'],
                        ['id_site','code_module','code_presentation'],
                      )

  clicks = clicks.drop(['id_site'], axis = 1)

  #clicks = clicks.groupby(['code_module','code_presentation','id_student','activity_type']).sum().reset_index()

  sum_click_df = pd.pivot_table(data = clicks_from(clicks,click_deadline), 
                              index = ['code_module','code_presentation','id_student'],
                              columns = 'activity_type', 
                              values = [f'sum_click{click_deadline} sum'],
                              fill_value = 0,
                              ).reset_index()

  # get rid of multi index
  sum_click_df = pd.concat([sum_click_df['code_module'],
                            sum_click_df['code_presentation'],
                            sum_click_df['id_student'], 
                            sum_click_df[f'sum_click{click_deadline} sum']], axis=1)

  temp = sum_click_df.groupby(['code_module','code_presentation','id_student']).sum(numeric_only=True, min_count=0)

  temp2 = pd.DataFrame()
  temp2['sum_click'] = temp.sum(axis=1)
  sum_click_df = pd.merge(sum_click_df,temp2,on = ['id_student','code_module','code_presentation'],how='left')

  return sum_click_df

In [ ]:
sum_click_df = get_click_data(student_vle_df.copy(),vle_df.copy(),CLICKS_DEADLINE)

### Merge sum_click_df and student_info_df

In [ ]:
student_all_info = merge_df(student_info_df.copy(), sum_click_df.copy(), on = ['id_student','code_module','code_presentation'],how='left')

student_all_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,sum_click
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,0.0,0.0,0.0,0.0,153.0,0.0,128.0,0.0,0.0,459.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,30.0,5.0,786.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,10.0,0.0,0.0,0.0,385.0,0.0,319.0,0.0,0.0,535.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,87.0,48.0,1396.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,0.0,0.0,0.0,0.0,105.0,0.0,59.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,22.0,4.0,260.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2.0,0.0,0.0,0.0,357.0,1.0,389.0,0.0,0.0,783.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,136.0,90.0,1772.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,0.0,0.0,0.0,0.0,149.0,4.0,179.0,0.0,0.0,461.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,65.0,11.0,909.0


In [ ]:
def tidy_up_dataset(student_all_info):
  # remove rows with null
  student_all_info.dropna(inplace=True)

  # replace final result string with numerical representation
  student_all_info.final_result = student_all_info.final_result.replace({'Withdrawn':0,'Fail':1,'Pass':2,'Distinction':3})

  # remove withdrawnn students
  student_all_info = student_all_info[student_all_info.final_result != 0]

  return student_all_info

In [ ]:
student_all_info = tidy_up_dataset(student_all_info.copy())

In [ ]:
student_all_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,dataplus,dualpane,externalquiz,folder,forumng,glossary,homepage,htmlactivity,oucollaborate,oucontent,ouelluminate,ouwiki,page,questionnaire,quiz,repeatactivity,resource,sharedsubpage,subpage,url,sum_click
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,2,0.0,0.0,0.0,0.0,153.0,0.0,128.0,0.0,0.0,459.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,30.0,5.0,786.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,2,10.0,0.0,0.0,0.0,385.0,0.0,319.0,0.0,0.0,535.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,87.0,48.0,1396.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,2,2.0,0.0,0.0,0.0,357.0,1.0,389.0,0.0,0.0,783.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,136.0,90.0,1772.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,2,0.0,0.0,0.0,0.0,149.0,4.0,179.0,0.0,0.0,461.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,65.0,11.0,909.0
5,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,2,7.0,0.0,0.0,0.0,723.0,4.0,455.0,0.0,0.0,848.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,89.0,43.0,2180.0


In [ ]:
student_all_info.code_module.value_counts()

BBB    5315
FFF    5026
DDD    3761
CCC    2250
GGG    2155
EEE    2078
AAA     601
Name: code_module, dtype: int64

In [ ]:
student_all_info.to_csv(f'oulad_dataset_combined_{DEADLINE}.csv', index=False)